<a href="https://colab.research.google.com/github/elibtronic/2025_state_of_scholarly_communication/blob/main/2025_APC_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# APC Analysis

How money was spent on APCs by Brock Researchers


In [1]:

import requests
import numpy as np
from pprint import pprint
import pandas as pd
from datetime import datetime
from google.colab import files

# @markdown ## OpenAlex Settings

# @markdown Enter in your email address for authentication to Open Alex
mailto = ""  # @param {type:"string"}
ror_id     = "056am2717"  # @param {type:"string"}
from_publication_date = "2024-07-01"  # @param {type:"date"}
to_publication_date = "2025-06-30"  # @param {type:"date"}

# @markdown ## Conversion between USD and CAD

usd_to_cad = 1.39 # @param {"type":"number","placeholder":"Conversation rate between USD and CAD"}



filter = f"authorships.institutions.ror:{ror_id},from_publication_date:{from_publication_date},to_publication_date:{to_publication_date},type:article,primary_location.source.type:journal"
api_url = f"https://api.openalex.org/works?filter={filter}&per-page=200&mailto={mailto}"

cursor = "*"  # Cursor for the first request
works = []  # List to accumulate all results

while cursor:
    # Construct the request URL with the current cursor
    request_url = api_url + f"&cursor={cursor}"

    # Get results from the API
    page = requests.get(request_url).json()
    results = page.get('results')
    meta = page.get('meta')

    # Accumulate results
    works += results

    # Get the next cursor from the current results
    cursor = meta.get('next_cursor')

    # Print progress
    total = int(meta.get("count"))
    #print(f"\r{len(works)}/{total}: {request_url}", end="")

# Do something with all results
print(f"\nTotal works: {len(works)}")


Total works: 670


In [2]:

apc_data_list = []
apc_paid_list  = []
apc_cad_list = []

for work in works:
  try:
    apc_list = work.get('apc_list')['value_usd']
  except:
    apc_list = np.nan

  try:
    apc_paid = work.get('apc_paid')['value_usd']
    apc_cad = work.get('apc_paid')['value_usd'] * usd_to_cad
  except:
    apc_paid = np.nan
    apc_cad = np.nan

  apc_data_list.append(apc_list)
  apc_paid_list.append(apc_paid)
  apc_cad_list.append(apc_cad)

apc_df = pd.DataFrame(zip(apc_data_list,apc_paid_list,apc_cad_list),columns=['APC','Amount Paid (USD)','Amount Paid (CAD)'])


In [3]:
apc_df.sample(5)

,APC,Amount Paid (USD),Amount Paid (CAD)
372,NaN,NaN,NaN
389,1450.0,1450.0,2015.50
55,NaN,NaN,NaN
430,NaN,NaN,NaN
622,2382.0,2382.0,3310.98


In [4]:
#total amount of APCS paid
apc_df['Amount Paid (CAD)'].sum()

np.float64(1027839.6699999999)

In [5]:
#total APC data retrieved
apc_df['APC'].sum() * usd_to_cad

np.float64(1489393.3399999999)

In [6]:
#mean paid APCs
apc_df['Amount Paid (CAD)'].mean()

np.float64(4111.358679999999)

In [7]:
#median paid APCS
apc_df['Amount Paid (CAD)'].median()

3912.85

In [8]:
#total of DOIs with APCs
apc_df['APC'].count()

np.int64(373)

In [9]:
#portion of DOIs with APCs
apc_df['APC'].count() / len(apc_df['APC'])

np.float64(0.5567164179104478)

In [10]:
#portion of DOIs that were paid
apc_df['Amount Paid (CAD)'].count()

np.int64(250)

In [11]:
#portion of paid APCs
apc_df['APC'].count() / len(apc_df['Amount Paid (CAD)'])

np.float64(0.5567164179104478)